In [57]:
import requests
from bs4 import BeautifulSoup
import json
import re
from concurrent.futures import ThreadPoolExecutor
import os
import pandas as pd
from tqdm import tqdm

### Extraindo o link da API dos produtos

In [15]:
workers = os.cpu_count()
categorias = ['medicamentos', 'saude-e-bem-estar', 'beleza-e-protecao', 'dermocosmeticos', 'mamae-e-bebe', 'higiene', 'primeiros-socorros', 'dieta-e-suplemento', 'aparelhos', 'perfumes', 'conveniencia']
headers = {
    "authority": "www.precopopular.com.br",
    "accept": "*/*",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "referer": "https://www.precopopular.com.br/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

lista_ids = []
cat_links = []

for categoria in categorias:
    url = f'https://www.precopopular.com.br/{categoria}?lid=1bcd623f-7892-409c-949f-2fe2aea069ff'
    response = requests.get(url, headers=headers)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Encontrar todas as tags <h4> com a classe específica da categoria
        tags_link = soup.find_all('h4', {'class': f'{categoria} even'})
        tags_id = soup.find_all('h4', {'class': f'{categoria} even'})
        for tagi in tags_id:
            link_categoria = tagi.find('a')['href']
            response_categoria = requests.get(link_categoria, headers=headers)
            if response_categoria.status_code == 200:
                match = re.search(r'"categoryId":"(\d+)".*?"departmentyId":"(\d+)"', response_categoria.text)
                # Verifique se a correspondência foi encontrada
                if match:
                    category_id = match.group(1)
                    department_id = match.group(2)
                    ids = {'department_id': department_id, 'category_id': category_id}
                    lista_ids.append(ids)
        for tagl in tags_link:
            link_categoria = tagl.find('a')['href']
            cat_links.append(link_categoria)
    else:
        print(f"Erro na requisição. Código de status: {response.status_code}")

df = pd.DataFrame({
    'category_ids': [ids['category_id'] for ids in lista_ids],
    'department_ids': [ids['department_id'] for ids in lista_ids],
    'links': cat_links})

In [43]:
links_produtos = []

for ids in lista_ids:
    department_id = ids['department_id']
    category_id = ids['category_id']

    # Iterar sobre as páginas
    page = 1
    while page <= 50:  # Defina um limite para evitar loops infinitos
        url = f'https://www.precopopular.com.br/buscapagina?fq=C:/{department_id}/{category_id}/&PS=24&sl=e29584ae-506a-4769-9290-5c3945b2357a&cc=24&sm=0&PageNumber={page}&O=OrderByTopSaleDESC'
        response = requests.get(url, headers=headers)

        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Parseia o conteúdo HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Encontrar os data-product-ids
            product_links = soup.select('.product-item__title a')
            if not product_links:
                break  # Sai do loop se não houver mais produtos

            for link in product_links:
                product_href = link['href'].split('/')[-2]
                info = {
                    'department_id': department_id,
                    'category_id': category_id,
                    'data_product_id': product_href
                }
                links_produtos.append(info)
            # Aumenta o número da página
            print(category_id, page)
            page += 1
        else:
            print(f"Erro na requisição. Código de status: {response.status_code}")
            break  # Sai do loop em caso de erro
df_links = pd.DataFrame(links_produtos)
df_links.to_csv('df_links.csv')

11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
12 1
390 1
390 2
390 3
390 4
390 5
390 6
390 7
390 8
390 9
390 10
390 11
390 12
390 13
390 14
390 15
390 16
390 17
390 18
390 19
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
13 15
13 16
13 17
13 18
13 19
13 20
13 21
13 22
13 23
13 24
13 25
13 26
13 27
13 28
13 29
13 30
13 31
30 1
30 2
30 3
30 4
30 5
30 6
30 7
30 8
30 9
30 10
30 11
30 12
30 13
30 14
30 15
30 16
30 17
30 18
41 1
41 2
41 3
41 4
41 5
41 6
41 7
41 8
41 9
41 10
41 11
48 1
48 2
48 3
48 4
48 5
48 6
48 7
48 8
48 9
391 1
391 2
391 3
391 4
391 5
391 6
391 7
391 8
391 9
391 10
391 11
391 12
391 13
391 14
391 15
391 16
391 17
391 18
391 19
391 20
391 21
391 22
391 23
391 24
391 25
391 26
391 27
391 28
391 29
391 30
391 31
391 32
391 33
391 34
391 35
391 36
391 37
391 38
391 39
391 40
391 41
391 42
391 43
391 44
391 45
391 46
391 47
391 48
391 49
391 50
49 1
49 2
49 3
49 4
49 5
49 6
50 1
50 2
51 1
51 2
51 3
51 4
51 5
51 6
51 7
51 8
51 9
51 10
51 11
51 12
51 1

### Extraindo as informações

In [58]:
def extrair_infos(href):
    try:
        api_url = f'https://www.precopopular.com.br/api/catalog_system/pub/products/search/{href}/p'
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            product_data_list = response.json()

            # Lista para armazenar todos os produtos encontrados
            produtos = []

            # Iterar sobre a lista de produtos
            for product_data in product_data_list:
                nome = product_data.get('productName', '')
                ean = product_data.get('items', [{}])[0].get('ean', '')
                marca = product_data.get('brand', '')
                
                descricao_raw = product_data.get('description', '')
                soup = BeautifulSoup(descricao_raw, 'html.parser')
                descricao_ = soup.get_text(separator=' ', strip=True)
                descricao = limpar_texto(descricao_)

                preco_s = product_data.get('items', [{}])[0].get('sellers', [{}])[0].get('commertialOffer', {}).get('ListPrice', 0.0)
                preco_c = product_data.get('items', [{}])[0].get('sellers', [{}])[0].get('commertialOffer', {}).get('Price', 0.0)
                porcentagem = round(100 * (1 - (preco_c / preco_s))) if preco_s > 0 else 0.0

                # Construir o dicionário com as informações e adicionar à lista
                info = {
                    'Produto': nome,
                    'EAN': ean,
                    'Marca': marca,
                    'Descrição': descricao,
                    'Preço sem desconto': preco_s,
                    'Preço com desconto': preco_c,
                    '% de desconto': f'{porcentagem}%',
                    'Farmácia': 'Preço Popular',
                    'Região': 'Sudeste',
                    'Cidade': 'São Paulo'
                }
                produtos.append(info)
                pbar.update(1)

            return produtos

        print(f"Nenhum produto encontrado para {href}")
        return None
    except Exception as e:
        print(f"Erro ao processar {href}: {str(e)}")
        return None
    
def limpar_texto(texto):
    # Substituir caracteres especiais de pontuação por espaços
    texto = re.sub(r'[^\w\s.,;:!?()\[\]{}-]', ' ', texto)
    # Substituir espaços múltiplos por um único espaço
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

# Criar uma lista para armazenar os resultados
infos = []

# Inicializar o tqdm
with tqdm(total=len(df_links)) as pbar:
    with ThreadPoolExecutor(max_workers=workers) as executor:
        # Usar o método map corretamente
        results = executor.map(extrair_infos, df_links['data_product_id'])
        
        # Filtrar resultados para remover valores nulos
        infos = [info for info in results if info is not None]

# Converter a lista de dicionários em um DataFrame do pandas
df_infos = pd.DataFrame([item[0] for item in infos])
df_infos.to_csv('df_infos.csv', index=False)

 19%|█▉        | 3332/17261 [08:44<34:55,  6.65it/s]  

Falha na requisição para lamictal-dispersivel-50mg-com-30-comprimidos. Código de status: 500


 63%|██████▎   | 10903/17261 [29:33<20:06,  5.27it/s] 

Falha na requisição para pente-marco-boni-com-2-1216. Código de status: 500


 70%|██████▉   | 12080/17261 [32:56<11:46,  7.33it/s]

...

 92%|█████████▏| 15870/17261 [43:25<04:01,  5.76it/s]

Falha na requisição para sabonete-protex-liquido-intimo-200ml-calm-<->-<->-protect-especial. Código de status: 400


 95%|█████████▌| 16463/17261 [44:56<02:10,  6.11it/s]

Falha na requisição para barra-bold-thin-40gr-caramelo-<->-<->-amendoim. Código de status: 400


100%|█████████▉| 17253/17261 [46:55<00:01,  6.13it/s]
